In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import shared

ve_or_ju = "ve"

In [2]:
running_order = pd.read_csv(f"data/running_order_2019_with_estimates_{ve_or_ju}.tsv", delimiter="\t")
running_order.head()

,Unnamed: 0,team_id,team,team_base_name,team_country,leg,leg_dist,name,leg_nro,orig_name,num_runs,pred_log_mean,pred_log_std,predicted,log_q_low,log_q_high,log_std,log_std_fixed,final_pace_mean,final_pace_std
0,0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,1,6.0,fiona bunn,1,Fiona Bunn,1.0,2.110,0.172,6.743671,1.863840,2.033659,0.084909,0.100000,2.110,0.100000
1,1,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,2,5.7,joana wälti,2,Joana Wälti,2.0,2.102,0.124,6.882701,1.809328,2.053610,0.122141,0.122141,2.102,0.122141
2,2,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,3,7.3,anna kindlundh,3,Anna Kindlundh,6.0,2.077,0.120,6.107582,1.726928,2.056589,0.164830,0.164830,2.077,0.120000
3,3,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,SWE,4,7.9,jeanette jönsson hellstadius,4,Jeanette Jönsson Hellstadius,3.0,2.041,0.134,7.474857,1.845341,2.089668,0.122164,0.122164,2.041,0.134000
4,4,68,Asker SK 1,Asker SK,NOR,1,6.0,synne strand,1,Synne Strand,1.0,2.139,0.121,8.184408,1.911360,2.173229,0.130934,0.130934,2.139,0.130934


In [3]:
order = running_order[["team_id", "team", "leg", "leg_dist", "orig_name", "final_pace_mean", "final_pace_std"]]


In [4]:
order = order.rename(index=str, columns={"orig_name": "name", "leg_dist": "dist", "final_pace_mean": "log_mean", "final_pace_std": "log_std"})

order.head()



,team_id,team,leg,dist,name,log_mean,log_std
0,8,Södertälje Nykvarn Orientering 1,1,6.0,Fiona Bunn,2.110,0.100000
1,8,Södertälje Nykvarn Orientering 1,2,5.7,Joana Wälti,2.102,0.122141
2,8,Södertälje Nykvarn Orientering 1,3,7.3,Anna Kindlundh,2.077,0.120000
3,8,Södertälje Nykvarn Orientering 1,4,7.9,Jeanette Jönsson Hellstadius,2.041,0.134000
4,68,Asker SK 1,1,6.0,Synne Strand,2.139,0.130934


In [5]:
by_teams = order.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                    \
leg                                     1                                 2   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
68                             Asker SK 1                        Asker SK 1   
80                          Freidig SPK 1                     Freidig SPK 1   
83       Södertälje Nykvarn Orientering 2  Södertälje Nykvarn Orientering 2   
128             Hollolan Urheilijat -46 1         Hollolan Urheilijat -46 1   

                                                                             \
leg                                     3                                 4   
team_id                                                                       
8        Södertälje Nykvarn Orientering 1  Södertälje Nykvarn Orientering 1   
68                             Asker SK 1                        Asker SK 1   
80                          Freidig SPK 1                     Freidig SPK 1   
83       Södertälje Nykvarn Orientering 2  Södertälje Nykvarn Orientering 2   
128             Hollolan Urheilijat -46 1         Hollolan Urheilijat -46 1   

        dist                         name                              \
leg        1    2    3    4             1                           2   
team_id                                                                 
8        6.0  5.7  7.3  7.9    Fiona Bunn                 Joana Wälti   
68       6.0  5.7  7.3  7.9  Synne Strand  Lotte Jauhojärvi Markussen   
80       6.0  5.7  7.3  7.9   Mari Eidsmo          Ane Malene Kolstad   
83       6.0  5.7  7.3  7.9    Laura King               Amanda Kleijn   
128      6.0  5.7  7.3  7.9  Kirsi Sipilä               Heidi Passila   

                                                            log_mean  \
leg                        3                             4         1   
team_id                                                                
8             Anna Kindlundh  Jeanette Jönsson Hellstadius  2.110000   
68       Anna Öfsthus Gravir                Marit Isachsen  2.139000   
80               Dina Reitan                  Vera Melkild  2.081351   
83               Sofia Wigle               Anna Ek Larsson  2.070204   
128          Matilda Suhonen                 Anneli Vainik  2.268000   

                                        log_std                                
leg             2         3         4         1         2         3         4  
team_id                                                                        
8        2.102000  2.077000  2.041000  0.100000  0.122141  0.120000  0.134000  
68       2.121000  2.274000  2.307000  0.130934  0.236187  0.182309  0.235000  
80       2.068919  2.090712  2.102000  0.167446  0.142361  0.159152  0.131051  
83       2.055800  2.060000  2.088466  0.137841  0.140509  0.160034  0.134402  
128      2.306000  2.499000  2.572000  0.133000  0.112000  0.171109  0.118923

In [6]:
display(by_teams.shape)
# remove teams that have not nominated runners for all legs
by_teams = by_teams[np.all(np.isfinite(by_teams["log_mean"]), axis=1)]
display(by_teams.shape)

(366, 20)

(358, 20)

In [7]:
num_legs = shared.num_legs[ve_or_ju]
# Sum of log normal variables is not defined 
# so we simulate 10000 runs for each user and sum them and then do statistics on simulated results 
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1, num_legs + 1):
        if np.isnan(row["log_mean"][i]):
            print(row["log_mean"])
            print(row["name"])
        samples[i] = row["dist"][i] * lognorm.rvs(s = row["log_std"][i], scale = np.exp(row["log_mean"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    # leg_1 
    # leg_1 + leg_2
    # leg_1 + leg_2 + leg_3
    # ...
    for i in range(1,num_legs + 1):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    sum_logs = np.log(samples_sums)
    sum_log_means = np.mean(sum_logs)
    sum_log_stds = np.std(sum_logs)
    
    """
    for i in range(1, num_legs + 1):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1, num_legs + 1):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_means_dict = {f"fin_sum_log_mean_{leg}" : sum_log_means.values[leg-1] for leg in range(1, num_legs + 1)}
    fin_sum_log_stds_dict = {f"fin_sum_log_std_{leg}" : sum_log_stds.values[leg-1] for leg in range(1, num_legs + 1)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict, **fin_sum_log_means_dict, **fin_sum_log_stds_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_end95_1,fin_end95_2,fin_end95_3,fin_end95_4,fin_median_1,fin_median_2,...,fin_mean_3,fin_mean_4,fin_sum_log_mean_1,fin_sum_log_mean_2,fin_sum_log_mean_3,fin_sum_log_mean_4,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,40.863359,82.882002,136.446885,192.143217,60.155254,112.544090,177.579324,244.218646,49.496410,96.462911,...,155.580304,216.793976,3.902461,4.569934,5.044914,5.377051,0.099481,0.078497,0.067032,0.061559
68,39.580318,76.891417,138.576305,205.541595,66.063622,130.670973,213.004783,310.388830,51.012609,99.177452,...,172.414866,253.547112,3.931978,4.599236,5.143903,5.529903,0.131713,0.134323,0.109483,0.106219
80,34.606070,75.731310,128.229400,188.672076,67.017315,116.903758,184.097224,253.859008,48.100869,93.691140,...,154.242559,219.431856,3.872238,4.540764,5.034259,5.388150,0.168398,0.111868,0.092352,0.076048
83,36.291490,76.289461,127.291465,186.933325,62.515749,112.655531,178.090683,247.330666,47.624566,92.794535,...,151.244301,215.562988,3.863697,4.528737,5.015214,5.370635,0.138429,0.099341,0.085842,0.072385
128,44.771782,97.801958,172.926956,269.337280,75.189589,137.461558,246.629440,357.556714,57.982686,115.620364,...,206.263582,310.499531,4.061206,4.751533,5.325179,5.735593,0.133237,0.086828,0.089043,0.071914


In [8]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()

,team_id,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,...,name_3,name_4,log_mean_1,log_mean_2,log_mean_3,log_mean_4,log_std_1,log_std_2,log_std_3,log_std_4
0,8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,...,Anna Kindlundh,Jeanette Jönsson Hellstadius,2.110000,2.102000,2.077000,2.041000,0.100000,0.122141,0.120000,0.134000
1,68,Asker SK 1,Asker SK 1,Asker SK 1,Asker SK 1,6.0,5.7,7.3,7.9,Synne Strand,...,Anna Öfsthus Gravir,Marit Isachsen,2.139000,2.121000,2.274000,2.307000,0.130934,0.236187,0.182309,0.235000
2,80,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,6.0,5.7,7.3,7.9,Mari Eidsmo,...,Dina Reitan,Vera Melkild,2.081351,2.068919,2.090712,2.102000,0.167446,0.142361,0.159152,0.131051
3,83,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,6.0,5.7,7.3,7.9,Laura King,...,Sofia Wigle,Anna Ek Larsson,2.070204,2.055800,2.060000,2.088466,0.137841,0.140509,0.160034,0.134402
4,128,Hollolan Urheilijat -46 1,Hollolan Urheilijat -46 1,Hollolan Urheilijat -46 1,Hollolan Urheilijat -46 1,6.0,5.7,7.3,7.9,Kirsi Sipilä,...,Matilda Suhonen,Anneli Vainik,2.268000,2.306000,2.499000,2.572000,0.133000,0.112000,0.171109,0.118923
5,131,Vittjärvs IK 1,Vittjärvs IK 1,Vittjärvs IK 1,Vittjärvs IK 1,6.0,5.7,7.3,7.9,Andrea Wickbom,...,Amanda Holmgren,Evelina Wickbom,2.053000,2.056000,2.043000,2.104000,0.114000,0.124000,0.137000,0.108000
6,141,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,Espoon Akilles 1,6.0,5.7,7.3,7.9,Essi Hjelm,...,Anna Voutilainen,Maija Heikkilä,2.220000,2.220255,2.043000,2.366000,0.155000,0.162653,0.112000,0.136000
7,143,Köping-Kolsva OK 1,Köping-Kolsva OK 1,Köping-Kolsva OK 1,Köping-Kolsva OK 1,6.0,5.7,7.3,7.9,Martina Welen,...,Rebecca Mårtensson,Malin Welen,2.110899,2.121000,2.176000,2.163543,0.147031,0.136000,0.136000,0.154598
8,152,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,Catalonia Team 1,6.0,5.7,7.3,7.9,Anna Serrallonga,...,Marta Bruns,Ona Rafols,2.343000,2.399240,2.597276,2.437731,0.171871,0.183999,0.193783,0.152054
9,157,Vehkalahden Veikot 1,Vehkalahden Veikot 1,Vehkalahden Veikot 1,Vehkalahden Veikot 1,6.0,5.7,7.3,7.9,Elina Kilpeläinen,...,Raita Pakkanen,Jenni Ovaska,2.096000,2.166000,2.093000,2.259000,0.093000,0.176310,0.125000,0.146000


In [9]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [10]:
estimates.head()

,team_1,team_2,team_3,team_4,dist_1,dist_2,dist_3,dist_4,name_1,name_2,...,fin_mean_3,fin_mean_4,fin_sum_log_mean_1,fin_sum_log_mean_2,fin_sum_log_mean_3,fin_sum_log_mean_4,fin_sum_log_std_1,fin_sum_log_std_2,fin_sum_log_std_3,fin_sum_log_std_4
team_id,,,,,,,,,,,,,,,,,,,,,
8,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering 1,6.0,5.7,7.3,7.9,Fiona Bunn,Joana Wälti,...,155.580304,216.793976,3.902461,4.569934,5.044914,5.377051,0.099481,0.078497,0.067032,0.061559
68,Asker SK 1,Asker SK 1,Asker SK 1,Asker SK 1,6.0,5.7,7.3,7.9,Synne Strand,Lotte Jauhojärvi Markussen,...,172.414866,253.547112,3.931978,4.599236,5.143903,5.529903,0.131713,0.134323,0.109483,0.106219
80,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,Freidig SPK 1,6.0,5.7,7.3,7.9,Mari Eidsmo,Ane Malene Kolstad,...,154.242559,219.431856,3.872238,4.540764,5.034259,5.388150,0.168398,0.111868,0.092352,0.076048
83,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,Södertälje Nykvarn Orientering 2,6.0,5.7,7.3,7.9,Laura King,Amanda Kleijn,...,151.244301,215.562988,3.863697,4.528737,5.015214,5.370635,0.138429,0.099341,0.085842,0.072385
128,Hollolan Urheilijat -46 1,Hollolan Urheilijat -46 1,Hollolan Urheilijat -46 1,Hollolan Urheilijat -46 1,6.0,5.7,7.3,7.9,Kirsi Sipilä,Heidi Passila,...,206.263582,310.499531,4.061206,4.751533,5.325179,5.735593,0.133237,0.086828,0.089043,0.071914


In [11]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_")
runner_estimates = runner_estimates.sort_values(by=['team_id', 'leg'])
#runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head(7)

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
68      1                          Asker SK 1   6.0   
        2                          Asker SK 1   5.7   
        3                          Asker SK 1   7.3   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.863359   
        2                     Joana Wälti     2.102  0.122141    82.882002   
        3                  Anna Kindlundh     2.077  0.120000   136.446885   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   192.143217   
68      1                    Synne Strand     2.139  0.130934    39.580318   
        2      Lotte Jauhojärvi Markussen     2.121  0.236187    76.891417   
        3             Anna Öfsthus Gravir     2.274  0.182309   138.576305   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.155254   49.496410   49.770105          3.902461   
        2    112.544090   96.462911   96.835749          4.569934   
        3    177.579324  155.172803  155.580304          5.044914   
        4    244.218646  216.385413  216.793976          5.377051   
68      1     66.063622   51.012609   51.452443          3.931978   
        2    130.670973   99.177452  100.315128          4.599236   
        3    213.004783  171.088835  172.414866          5.143903   

             fin_sum_log_std  
team_id leg                   
8       1           0.099481  
        2           0.078497  
        3           0.067032  
        4           0.061559  
68      1           0.131713  
        2           0.134323  
        3           0.109483

In [12]:
runner_estimates[runner_estimates['team'].str.contains("Reak")][["name", "fin_start95", "fin_median", "fin_end95"]]

,,name,fin_start95,fin_median,fin_end95
team_id,leg,,,,


In [13]:
# Convert minutes to date and times
start_timestamp = shared.start_timestamp[ve_or_ju][2019]

runner_estimates["fin_time_median"] = pd.to_datetime(runner_estimates["fin_median"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_start95"] = pd.to_datetime(runner_estimates["fin_start95"] * 60, unit = "s", origin= start_timestamp)
runner_estimates["fin_time_end95"] = pd.to_datetime(runner_estimates["fin_end95"] * 60, unit = "s", origin= start_timestamp)

runner_estimates.head()

team  dist  \
team_id leg                                           
8       1    Södertälje Nykvarn Orientering 1   6.0   
        2    Södertälje Nykvarn Orientering 1   5.7   
        3    Södertälje Nykvarn Orientering 1   7.3   
        4    Södertälje Nykvarn Orientering 1   7.9   
68      1                          Asker SK 1   6.0   

                                     name  log_mean   log_std  fin_start95  \
team_id leg                                                                  
8       1                      Fiona Bunn     2.110  0.100000    40.863359   
        2                     Joana Wälti     2.102  0.122141    82.882002   
        3                  Anna Kindlundh     2.077  0.120000   136.446885   
        4    Jeanette Jönsson Hellstadius     2.041  0.134000   192.143217   
68      1                    Synne Strand     2.139  0.130934    39.580318   

              fin_end95  fin_median    fin_mean  fin_sum_log_mean  \
team_id leg                                                         
8       1     60.155254   49.496410   49.770105          3.902461   
        2    112.544090   96.462911   96.835749          4.569934   
        3    177.579324  155.172803  155.580304          5.044914   
        4    244.218646  216.385413  216.793976          5.377051   
68      1     66.063622   51.012609   51.452443          3.931978   

             fin_sum_log_std               fin_time_median  \
team_id leg                                                  
8       1           0.099481 2019-06-15 14:49:29.784604311   
        2           0.078497 2019-06-15 15:36:27.774649858   
        3           0.067032 2019-06-15 16:35:10.368187189   
        4           0.061559 2019-06-15 17:36:23.124801874   
68      1           0.131713 2019-06-15 14:51:00.756511450   

                         fin_time_start95                fin_time_end95  
team_id leg                                                              
8       1   2019-06-15 14:40:51.801535368 2019-06-15 15:00:09.315213442  
        2   2019-06-15 15:22:52.920115232 2019-06-15 15:52:32.645381212  
        3   2019-06-15 16:16:26.813104868 2019-06-15 16:57:34.759454489  
        4   2019-06-15 17:12:08.593013763 2019-06-15 18:04:13.118735075  
68      1   2019-06-15 14:39:34.819103241 2019-06-15 15:06:03.817338705

In [14]:

for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(2)
for_print["fin_time_median"] = for_print["fin_time_median"].dt.strftime("%H:%M")
for_print["fin_time_start95"] = for_print["fin_time_start95"].dt.strftime("%H:%M")
for_print["fin_time_end95"] = for_print["fin_time_end95"].dt.strftime("%H:%M")
for_print = for_print[['team',
 'leg',
 'name', 
 'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_print[for_print['team'].str.contains("Reak")]

,team,leg,name,fin_time_start95,fin_time_end95,fin_time_median
team_id,,,,,,


In [15]:
for_web = runner_estimates.copy().reset_index()
for_web["fin_time_median"] = for_web["fin_time_median"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.tz_localize('EET').dt.tz_convert('UTC')
for_web["ind_log_mean"] = for_web["log_mean"]
for_web["ind_log_std"] = for_web["log_std"]

for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.tz_localize('EET').dt.tz_convert('UTC')

#for_web["fin_time_median"] = for_web["fin_time_median"].dt.strftime("%H:%M")
#for_web["fin_time_start95"] = for_web["fin_time_start95"].dt.strftime("%H:%M")
#for_web["fin_time_end95"] = for_web["fin_time_end95"].dt.strftime("%H:%M")
for_web = for_web[[
 'team_id',
 'leg',
 'team',
 'name',
 'ind_log_mean', 
 'ind_log_std',
 'fin_sum_log_mean', 
 'fin_sum_log_std',
  'fin_time_start95', 
 'fin_time_end95',
 'fin_time_median']]

for_web[for_web['team'].str.contains("Jyry")]
#for_web[for_web['team'].str.contains("Reak")]

,team_id,leg,team,name,ind_log_mean,ind_log_std,fin_sum_log_mean,fin_sum_log_std,fin_time_start95,fin_time_end95,fin_time_median
140,341,1,Rasti-Jyry 1,Jola Maisala,2.197000,0.117000,3.990557,0.115866,2019-06-15 11:43:19.008654118+00:00,2019-06-15 12:07:51.978340626+00:00,2019-06-15 11:54:02.489337444+00:00
141,341,2,Rasti-Jyry 1,Kaija Rantoja,2.364000,0.140000,4.746903,0.093216,2019-06-15 12:35:44.784252882+00:00,2019-06-15 13:18:33.815571785+00:00,2019-06-15 12:55:10.814379692+00:00
142,341,3,Rasti-Jyry 1,Jenni Castren,2.395000,0.152000,5.279344,0.084071,2019-06-15 13:46:44.541264772+00:00,2019-06-15 14:51:58.789090872+00:00,2019-06-15 14:16:12.472802639+00:00
143,341,4,Rasti-Jyry 1,Inka Lemmetyinen,2.558000,0.178309,5.702416,0.082287,2019-06-15 15:15:08.552918196+00:00,2019-06-15 16:54:03.154339075+00:00,2019-06-15 15:59:05.621230125+00:00
264,528,1,Rasti-Jyry 2,Päivi Taponen,2.418000,0.163000,4.208978,0.164422,2019-06-15 11:48:46.612825871+00:00,2019-06-15 12:33:02.192029715+00:00,2019-06-15 12:07:16.939296722+00:00
265,528,2,Rasti-Jyry 2,Miia Hovi,2.229000,0.141000,4.796646,0.111145,2019-06-15 12:37:22.667107821+00:00,2019-06-15 13:30:37.943417072+00:00,2019-06-15 13:01:07.252952099+00:00
266,528,3,Rasti-Jyry 2,Mervi Luurila,2.459000,0.191317,5.335094,0.103136,2019-06-15 13:49:44.261018276+00:00,2019-06-15 15:15:04.199362993+00:00,2019-06-15 14:27:27.132980585+00:00
267,528,4,Rasti-Jyry 2,Tiina Isoniemi,2.371000,0.116000,5.678854,0.080282,2019-06-15 15:10:08.727984667+00:00,2019-06-15 16:43:44.991762638+00:00,2019-06-15 15:52:36.181301355+00:00
396,740,1,Rasti-Jyry 4,Mila Viksilä,2.581289,0.144432,4.373734,0.143217,2019-06-15 12:00:05.821231365+00:00,2019-06-15 12:45:01.533041954+00:00,2019-06-15 12:19:17.828676701+00:00
397,740,2,Rasti-Jyry 4,Tuula Ohramaa,2.617000,0.174000,5.064642,0.112520,2019-06-15 13:06:46.449559450+00:00,2019-06-15 14:17:40.597650528+00:00,2019-06-15 13:38:24.132898808+00:00


In [16]:
for_web.to_json(f'web-lib/for_web_{ve_or_ju}_2019.json', orient="records", date_format="iso")